# Example Code to test the Pub_worm API

In [ ]:
import sys
import inspect
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter
import os
import json

# Add pub_worm directory to the Python path
sys.path.insert(0, "/Users/dan/Code/Python/pub_worm")

from pub_worm.ncbi.entreze_api import EntrezAPI

# Find where EntrezAPI is being load from
module = inspect.getmodule(EntrezAPI)
if hasattr(module, "__file__"):
    file_path = module.__file__
    print("EntrezAPI imported from:", file_path)
else:
    print("Could not determine the file path.")

In [ ]:
# Example use entreze_elink_pmid_to_pmcid to get pub med central id for full text pull
# First do an esearch, with the results pmids get the pmcids with entreze_elink_pmid_to_pmcid
# With the pmcids use entreze_efetch to get the full article data

ncbi_api = EntrezAPI()
search_term = "Marnett L[au] AND (2019/01/01:2024/04/16[pdat])AND (vanderbilt[affil])"
esearch_params = {'term': search_term }
esearch_result = ncbi_api.entreze_esearch(esearch_params)


elink_result = ncbi_api.entreze_elink_pmid_to_pmcid(esearch_result)

params= {'db': 'pmc'}
epost_result = ncbi_api.entreze_epost(elink_result, params)

efetch_result = ncbi_api.entreze_efetch(epost_result)

efetch_result_df = pd.DataFrame(efetch_result['articles'])
for index, row in efetch_result_df.iterrows():
    pmcid = row['pmcid']
    body = row['body']
    filename = f"output/{pmcid}.txt"
    
    # Write body content to file
    with open(filename, 'w') as file:
        file.write(body)

efetch_result_df = efetch_result_df.drop('body', axis=1)
efetch_result_df.to_csv("output/pmc_articles.csv",index=False)
efetch_result_df

In [ ]:
# Example call to ePost and eSummary
data = ["10021351", "10022905", "10022914", "10022975", "10048487", "10048958", "10049162", "10049362", "10049567", "10049576", "10051671", "10051850", "10064800", "10066248"]

ncbi_api = EntrezAPI()
entreze_epost_result = ncbi_api.entreze_epost(data)

if 'WebEnv' in entreze_epost_result:
    entreze_summaries_result = ncbi_api.entreze_pmid_summaries(entreze_epost_result)
    pretty_data = json.dumps(entreze_summaries_result, indent=4)
    print(pretty_data)

In [ ]:
# Example code to Get full Articles from entrez/eutils
import requests
import time

def fetch_full_article(pmcid):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        "db": "pmc",
        "id": pmcid,
        "retmode": "xml"
    }

    response = requests.get(base_url, params=params)
    time.sleep(2)

    if response.status_code == 200:
        print(f"response.status_code {response.status_code}")
        return response.text
    else:
        print(f"Failed to fetch article for PMID {pmcid}. Status code: {response.status_code}")
        return None

# Example usage
pmcids = ["3549060"]  # Replace with the PMCID of the article you want to fetch
#pmids = ["10021351", "10022905", "10022914", "10022975", "10048487", "10048958", "10049162", "10049362", "10049567", "10049576", "10051671", "10051850", "10064800", "10066248"]
for pmcid in pmcids:
    article_xml = fetch_full_article(pmcid)
    if article_xml:
        with open(f"./output/article_{pmcid}.xml", "w") as file:
            file.write(article_xml)


In [ ]:
import os
env_vars = os.environ

# Print each environment variable
for key, value in env_vars.items():
    print(f"{key}: {value}")

In [ ]:
# Example of pulling 375 Record and iterating 200 at a time
ncbi_api = EntrezAPI()
search_term = "Walker A[au] AND (2023/01/01:2024/04/16[pdat])"
esearch_params = {'term': search_term }
esearch_result = ncbi_api.entreze_esearch(esearch_params)

efetch_result = ncbi_api.entreze_efetch(esearch_result)
efetch_result


In [ ]:
ncbi_api = EntrezAPI()
data = ["38632828"]
interm_result = ncbi_api.entreze_epost(data)

actual_result = ncbi_api.entreze_efetch(interm_result)
actual_result

In [ ]:
len(actual_result['authors'])